# Comparing AstroStandards Coordinate X-forms to Astropy

Kerry N. Wood

kerry.wood@asterism.ai

January 22, 2023

- code is developed to compare the coordinate transforms in the AstroStandards vs. AstroPy
- once we're happy, I move those into a library so we can use them elsewhere (`as_coordinates.py`)

In [ ]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

import astropy.coordinates
import astropy.units as u
import astropy.time

import matplotlib.pyplot as plt

# load our astrostandards harnesses and helper code
from load_utils import *
import helpers

In [ ]:
# init the astrostandards
init_all()

In [ ]:
# we need time and coordinate (nutation) config.  
# I wrote code that translates finals_2000A to the astrostandards format at : https://github.com/kerrywood/final2000_time_constants.git
TimeFuncDll.TimeFuncLoadFile( Cstr('./time-constants.dat', 512) )

## Testing process

- generate a set of points that are assumed to be in a frame
- convert using utilities from AstroStandards and AstroPy
- compare the answers

In [ ]:
N = 10000

test_EPH = np.random.uniform(6500, 55000, size=(N,3) )
# go from 2000-2025 and randomly sample
test_time = [  datetime(year=2000,month=1,day=1) + timedelta(days=X) for X in np.random.uniform(0,25*365.25,size=N) ]
astro_date = astropy.time.Time( test_time )

## GCRS (J2K) to TEME

In [ ]:
# ------------------------------------  ASTROPY  ------------------------------------
# astropy data
astro_gcrs = astropy.coordinates.GCRS( x = test_EPH[:,0] * u.km,
                                       y = test_EPH[:,1] * u.km,
                                       z = test_EPH[:,2] * u.km,
                                       obstime = astro_date,
                                       representation_type = 'cartesian')
# astropy GCRS to TEME
astro_teme = astro_gcrs.transform_to( astropy.coordinates.TEME( obstime=test_time ) ).cartesian.xyz.to_value(u.km).T


# ------------------------------------ ASTRO STANDARDS ------------------------------------
# convert dates to astrostandards format using helpers
as_dates = [ helpers.datetime_to_ds50(X,TimeFuncDll) for X in test_time ]

# astrostandards GCRS to TEME
vel_     = (ctypes.c_double*3)(0,0,0)
posteme  = (ctypes.c_double*3)()
velteme  = (ctypes.c_double*3)()

def gcrs2teme_row( pos,date ):
    AstroFuncDll.RotJ2KToDate( 1, 106, date, (ctypes.c_double * 3)(*pos), vel_, posteme, velteme )
    return np.array( posteme )

def gcrs2teme( mat, dates ):
    return np.vstack( [gcrs2teme_row(X,d) for X,d in zip(mat,dates)] )

as_teme = gcrs2teme( test_EPH, as_dates )

In [ ]:
err = as_teme - astro_teme
plt.close('all')
_ = plt.hist( np.linalg.norm(err, axis=1 ), bins=100, color='#818181')
plt.grid()
plt.xlabel("Difference (km)")
plt.ylabel('Count')
_ = plt.title('Astropy/AstroStandards GCRS->TEME comparison')

## TEME to GCRS (J2K) 

In [ ]:
# ------------------------------------  ASTROPY  ------------------------------------
# astropy data
astro_teme = astropy.coordinates.TEME( x = test_EPH[:,0] * u.km,
                                       y = test_EPH[:,1] * u.km,
                                       z = test_EPH[:,2] * u.km,
                                       obstime = astro_date,
                                       representation_type = 'cartesian')
# astropy GCRS to TEME
astro_gcrs = astro_teme.transform_to( astropy.coordinates.GCRS( obstime=test_time ) ).cartesian.xyz.to_value(u.km).T

# ------------------------------------ ASTRO STANDARDS ------------------------------------
# convert dates to astrostandards format using helpers
as_dates = [ helpers.datetime_to_ds50(X,TimeFuncDll) for X in test_time ]

# astrostandards GCRS to TEME
vel_   = (ctypes.c_double*3)(0,0,0)
posj2k = (ctypes.c_double*3)()
velj2k = (ctypes.c_double*3)()

def teme2gcrs_row( pos,date ):
    AstroFuncDll.RotDateToJ2K( 1, 106, date, (ctypes.c_double * 3)(*pos), vel_, posj2k, velj2k )
    return np.array( posj2k )

def teme2gcrs( mat, dates ):
    return np.vstack( [teme2gcrs_row(X,d) for X,d in zip(mat,dates)] )

as_gcrs = teme2gcrs( test_EPH, as_dates )

In [ ]:
err = as_gcrs - astro_gcrs
plt.close('all')
_ = plt.hist( np.linalg.norm(err, axis=1 ), bins=100, color='#818181')
plt.grid()
plt.xlabel("Difference (km)")
plt.ylabel('Count')
_ = plt.title('Astropy/AstroStandards TEME->GCRS comparison')

## ECI to EFG

- EFG in AstroStandards is a little confusing..
- note that in AstroStd, ECI means TEME

In [ ]:
# ------------------------------------  ASTROPY  ------------------------------------
# astropy data
astro_teme = astropy.coordinates.TEME( x = test_EPH[:,0] * u.km,
                                       y = test_EPH[:,1] * u.km,
                                       z = test_EPH[:,2] * u.km,
                                       obstime = astro_date,
                                       representation_type = 'cartesian')
# astropy GCRS to TEME
astro_itrs = astro_teme.transform_to( astropy.coordinates.ITRS( obstime=astro_date ) ).cartesian.xyz.to_value(u.km).T

# ------------------------------------ ASTRO STANDARDS ------------------------------------
# convert dates to astrostandards format using helpers
as_dates = [ helpers.datetime_to_ds50(X,TimeFuncDll) for X in test_time ]

# astrostandards GCRS to TEME
vel_   = (ctypes.c_double*3)(0,0,0)
posefg = (ctypes.c_double*3)()
velefg = (ctypes.c_double*3)()

def teme2efg_row( pos,date ):
    AstroFuncDll.ECIToEFGTime( date, (ctypes.c_double * 3)(*pos), vel_, posefg, velefg )
    return np.array( posefg )

def teme2efg( mat, dates ):
    return np.vstack( [teme2efg_row(X,d) for X,d in zip(mat,dates)] )

as_efg = teme2efg( test_EPH, as_dates )

In [ ]:
err = as_efg - astro_itrs
plt.close('all')
_ = plt.hist( np.linalg.norm(err, axis=1 ), bins=100, color='#818181', log=True)
plt.grid()
plt.xlabel("Difference (km)")
plt.ylabel('Count')
_ = plt.title('Astropy/AstroStandards TEME->ITRS/EFG comparison')

## EFG to ECI

In [ ]:
# ------------------------------------  ASTROPY  ------------------------------------
# astropy data
astro_itrs = astropy.coordinates.ITRS( x = test_EPH[:,0] * u.km,
                                       y = test_EPH[:,1] * u.km,
                                       z = test_EPH[:,2] * u.km,
                                       obstime = astro_date,
                                       representation_type = 'cartesian')
# astropy GCRS to TEME
astro_teme = astro_itrs.transform_to( astropy.coordinates.TEME( obstime=astro_date ) ).cartesian.xyz.to_value(u.km).T

# ------------------------------------ ASTRO STANDARDS ------------------------------------
# convert dates to astrostandards format using helpers
as_dates = [ helpers.datetime_to_ds50(X,TimeFuncDll) for X in test_time ]

# astrostandards GCRS to TEME
vel_   = (ctypes.c_double*3)(0,0,0)
posteme = (ctypes.c_double*3)()
velteme = (ctypes.c_double*3)()

def efg2teme_row( pos,date ):
    AstroFuncDll.EFGToECITime( date, (ctypes.c_double * 3)(*pos), vel_, posteme, velteme )
    return np.array( posteme )

def efg2teme( mat, dates ):
    return np.vstack( [efg2teme_row(X,d) for X,d in zip(mat,dates)] )

as_efg = efg2teme( test_EPH, as_dates )

In [ ]:
err = as_efg - astro_teme
plt.close('all')
_ = plt.hist( np.linalg.norm(err, axis=1 ), bins=100, color='#818181', log=True)
plt.grid()
plt.xlabel("Difference (km)")
plt.ylabel('Count')
_ = plt.title('Astropy/AstroStandards GCRS->TEME comparison')

## TEME to LLH

In [ ]:
# ------------------------------------  ASTROPY  ------------------------------------
# astropy data
astro_teme = astropy.coordinates.TEME( x = test_EPH[:,0] * u.km,
                                       y = test_EPH[:,1] * u.km,
                                       z = test_EPH[:,2] * u.km,
                                       obstime = astro_date,
                                       representation_type = 'cartesian')
# astropy GCRS to TEME
astro_itrs = astro_teme.transform_to( astropy.coordinates.ITRS( obstime=astro_date ) )
astro_el   = astro_itrs.earth_location.to_geodetic('WGS84')
astro_llh  = np.vstack( (astro_el.lat.to_value(u.deg),astro_el.lon.to_value(u.deg),astro_el.height.to_value(u.km) ) ).T

# ------------------------------------ ASTRO STANDARDS ------------------------------------
# convert dates to astrostandards format using helpers
as_dates = [ helpers.datetime_to_ds50(X,TimeFuncDll) for X in test_time ]

# astrostandards GCRS to TEME
vel_   = (ctypes.c_double*3)(0,0,0)
llh    = (ctypes.c_double*3)()

def eci2llh_row( pos,date ):
    AstroFuncDll.XYZToLLHTime( date, (ctypes.c_double * 3)(*pos), llh )
    return np.array( llh )

def eci2llh( mat, dates ):
    return np.vstack( [eci2llh_row(X,d) for X,d in zip(mat,dates)] )

as_llh = eci2llh( test_EPH, as_dates )

In [ ]:
err = as_llh - astro_llh
plt.close('all')
f,ax = plt.subplots(1,3,figsize=(15,5), sharey=True)
ax[0].plot( as_llh[:,0] - astro_llh[:,0],'.', color='k', markersize=1.0)
err2 =  as_llh[:,1] - astro_llh[:,1]
err2[ np.abs(err2) > 180 ] = 0
ax[1].plot( err2,'.', color='k', markersize=1.0)
ax[2].plot( as_llh[:,2] - astro_llh[:,2],'.', color='k', markersize=1.0)
for a in ax: a.grid()
ax[0].set_title('Lat error -- TEME->LLH')
ax[1].set_title('Lon error -- TEME->LLH')
ax[2].set_title('Alt error -- TEME->LLH')

# Topocentric (using only AstroFunc)

- the astrostandards open `AstroFunc.h` has routines to help with topocentric computations, but they are wonky
- this mega-function comuptes lots of attributes that you might need for sensor tasking / windows
- peel out what you need

- it builds a dataframe for a sensor site with all the necessary coordinates to find look vectors to targets
- if you have a space-based sensor, skip the LLH conversion and just set the coordinate fields

In [ ]:
def setup_times( ds50_utc : float) :    # date in astrostandards epoch 
    '''
    this will convert a set of astrostandard formatted times in UTC
    to other time formats needed for conversion later
    this is often the first function run
    '''
    def convert_from_utc( d50u ) :
        d50et = TimeFuncDll.UTCToET( d50u )
        d50ut = TimeFuncDll.UTCToUT1( d50u )
        theta = TimeFuncDll.ThetaGrnwchFK5( d50ut )
            
        return {'ds50_utc' : d50u,
                'ds50_et'  : d50et,
                'ds50_ut1' : d50ut,
                'sidereal' : theta}
    return pd.DataFrame( [ convert_from_utc(D) for D in ds50_utc ] )

# setup_times( as_dates )

In [ ]:
def llh_to_eci(      times : pd.DataFrame,     # from setup_times
                     lat : float,              # latitude (degrees)
                     lon : float,              # longitude (degrees)
                     alt : float):             # altitude (above WGS84; km)
    '''
    times must have the ds50_utc entry filled
    '''
    # constant longitude
    utc_dates      = times['ds50_utc']
    sensor_lon_rad = np.radians(lon)
    sen_eci        = (ctypes.c_double * 3)()
    llh            = (ctypes.c_double * 3)( lat, lon, alt )
    def llhtoxyz( d50_utc ):
        theta = TimeFuncDll.ThetaGrnwchFK5( d50_utc )
        lst   = sensor_lon_rad + theta
        AstroFuncDll.LLHToXYZTime( d50_utc, llh, sen_eci )
        return {'lat' : float(llh[0]), 'lon' : float(llh[1]), 'alt' : float(llh[2]), 
                'theta'  : theta, 'lst' : lst,
                'teme_p' : list( sen_eci ),
                'teme_v' : np.zeros(3) }
    return pd.concat( (times, pd.DataFrame( [llhtoxyz(D) for D in utc_dates ])), axis=1)

# llh_to_eci(setup_times(as_dates), 0, 90, 0)

In [ ]:
def get_celest( times : pd.DataFrame,      # from setup_times
                fcn   : callable ):       
    '''
    times must have the ds50_et entry filled
    '''
    sun_v  = (ctypes.c_double * 3)()
    sun_m  = ctypes.c_double()
    def getpos( D ): 
        fcn( D, sun_v, sun_m )
        return {'teme_p' : float(sun_m.value) * np.array(sun_v), 
               'teme_v' : np.zeros(3)}
    return pd.DataFrame( [getpos(X) for X in times['ds50_et'] ] )

def get_sun(   times: pd.DataFrame ):
    return get_celest( times, AstroFuncDll.CompSunPos )

def get_moon(   times: pd.DataFrame ):
    return get_celest( times, AstroFuncDll.CompMoonPos )
    
# get_celest( setup_times(as_dates), AstroFuncDll.CompSunPos )
# get_sun( setup_times(as_dates ) )
# get_moon( setup_times(as_dates ) )

In [ ]:
def topographic_calcs( observer_df : pd.DataFrame,      # must have times and teme locations filled in
                       target_df   : pd.DataFrame ) :   # must have tmies and teme locations filled in
    '''
    you must have local sidereal time (lst) in this frame (so even for space based,
    you need this value
    '''
    TOPO = helpers.astrostd_named_fields( AstroFuncDll, prefix='XA_TOPO_' )
    def getTopo( S, T ):
        AstroFuncDll.ECIToTopoComps( S['lst'], 
                                     S['lat'], 
                                     (ctypes.c_double * 3)(*S['teme_p']), 
                                     (ctypes.c_double * 3)(*T['teme_p']), 
                                     (ctypes.c_double * 3)(*T['teme_v']), TOPO.data )
        return TOPO.toDict()

    topo = pd.DataFrame( [getTopo(A,B) for A,B in zip( observer_df.to_dict('records'), target_df.to_dict('records')) ] )
    return pd.concat( (observer_df.add_suffix('_sensor'), target_df.add_suffix('target'), topo ), axis=1 )

# this is the recipe to get the moon
times = setup_times( as_dates )
sen   = llh_to_eci( times, 0, 90, 0 )
moon  = get_moon( times )
test_topo = topographic_calcs( sen, moon )

In [ ]:
test_topo.columns

In [ ]:
a_moon = astropy.coordinates.get_body('moon', astro_date)
a_sen  = astropy.coordinates.EarthLocation( lat=0*u.deg, lon=90*u.deg, height=0*u.km)
a_look = a_moon.transform_to( astropy.coordinates.AltAz( location=a_sen, obstime=astro_date ) )

In [ ]:
err = as_llh - astro_llh
plt.close('all')
f,ax = plt.subplots(1,3,figsize=(15,5), sharey=True )
azerr = test_topo['XA_TOPO_AZ'] - a_look.az.to_value(u.deg)
elerr = test_topo['XA_TOPO_EL'] - a_look.alt.to_value(u.deg)
rnerr = test_topo['XA_TOPO_RANGE'] - a_look.distance.to_value(u.km)
azerr = azerr[ np.abs(azerr) < 0.2 ]
# ax[0].plot( azerr,'.', color='r', markersize=2.0)
ax[0].hist( azerr, bins=100,  color='#818181')
ax[1].hist( elerr, bins=100,  color='#818181')
ax[2].hist( rnerr, bins=100,  color='#818181')
for a in ax: a.grid()
ax[0].set_title('Az error : Moon from 0,90')
ax[1].set_title('El error : Moon from 0,90')
ax[2].set_title('Range error : Moon from 0,90 (km)')
# ax[0].set_xlim((-0.2,0.2))